# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('./customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [8]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [11]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [13]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [28]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.0,0.015873,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012987,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012987,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.015385,0.015385,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014925,0.0
412,0.0,0.0,0.0,0.0,0.013699,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.013699,0.013699,0.013699,0.0,0.0,0.0,0.000000,0.000000,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [29]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

#para encontrar los clientes q se parece mas a uno en particular (eliminamos la primera pq es 1, ya que compara al cliente con el mismo)
#print(cust_dist.loc[33].shift(-1).sort_values(ascending=False))

#print(cust_dist.loc[33].shift(-1).max())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [24]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique()) #unique elimina los repetidos

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [25]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [26]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [27]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [34]:
prod_dist_cos = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'cosine'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

prod_dist_cos.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.525261,0.526165,0.555104,0.564050,0.519332,0.536893,0.517640,0.547829,0.575260,...,0.537868,0.550524,0.549926,0.525350,0.517278,0.550767,0.532645,0.534287,0.545650,0.523806
"Appetizer - Mini Egg Roll, Shrimp",0.525261,1.000000,0.527240,0.529561,0.544405,0.520697,0.527755,0.556529,0.551599,0.533705,...,0.547988,0.541984,0.550369,0.525597,0.525008,0.515063,0.537960,0.548444,0.536942,0.538182
Appetizer - Mushroom Tart,0.526165,0.527240,1.000000,0.545139,0.516876,0.528529,0.529274,0.531369,0.542411,0.517489,...,0.539689,0.528103,0.531201,0.552545,0.540265,0.545464,0.540018,0.539135,0.540934,0.526612
Appetizer - Sausage Rolls,0.555104,0.529561,0.545139,1.000000,0.544081,0.550881,0.526215,0.535372,0.526842,0.536206,...,0.538307,0.549571,0.548994,0.565185,0.530477,0.513588,0.536583,0.532383,0.531658,0.527799
Apricots - Dried,0.564050,0.544405,0.516876,0.544081,1.000000,0.538834,0.531543,0.543368,0.532270,0.544630,...,0.534045,0.540305,0.553428,0.550042,0.517577,0.511678,0.547295,0.542470,0.539973,0.550194


### Step 2: Get the products purchased for a specific customer of your choice.

In [32]:
prod_cust_pivot[33].sort_values(ascending=False).head()

ProductName
Grouper - Fresh                0.001229
Veal - Inside, Choice          0.001098
Lettuce - Spring Mix           0.001012
Salsify, Organic               0.000906
Water - Green Tea Refresher    0.000756
Name: 33, dtype: float64

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [38]:
for customer in cust_prod_pivot.index:
    top_prods_by_customer = prod_cust_pivot[customer].sort_values(ascending=False).head()
    top_related_product_for_product = pd.Series([])
    for product in top_prods_by_customer.index:
        top_related_product_for_product = pd.concat([top_related_product_for_product,prod_dist_cos[product].sort_values(ascending=False)])
        top_related_product_for_product = top_related_product_for_product.sort_values(ascending=False)
    print(pd.Series(list(top_related_product_for_product.index.unique())))

0                        Grouper - Fresh
1            Water - Green Tea Refresher
2                  Veal - Inside, Choice
3                       Salsify, Organic
4                   Lettuce - Spring Mix
5                 Liners - Banana, Paper
6               Table Cloth 62x114 White
7                Meldea Green Tea Liquor
8                         Smoked Paprika
9                     Pork - Belly Fresh
10                        Fond - Neutral
11                          Jagermeister
12                         Lamb - Ground
13                Cheese - Cheddarsliced
14                       Skirt - 29 Foot
15           Beets - Candy Cane, Organic
16            Soup V8 Roasted Red Pepper
17                           Beans - Wax
18                     Apricots - Halves
19             Beef - Texas Style Burger
20                 Puree - Passion Fruit
21                     Beef - Short Loin
22                     Butter - Unsalted
23                            Watercress
24         Pastr

0             Smirnoff Green Apple Twist
1                       Mussels - Frozen
2              Ice Cream Bar - Drumstick
3                             Watercress
4                          Cumin - Whole
5          Wine - Hardys Bankside Shiraz
6         Chocolate - Semi Sweet, Calets
7                  Hickory Smoke, Liquid
8                   Garbage Bags - Clear
9                  Cookie Dough - Double
10                     Beef - Short Loin
11     Wine - Vineland Estate Semi - Dry
12        Bread - Roll, Soft White Round
13               Cheese - Parmesan Cubes
14           Mayonnaise - Individual Pkg
15                     Juice - V8 Splash
16                      Onions - Vidalia
17         Wine - Ej Gallo Sierra Valley
18            Table Cloth - 53x69 Colour
19                   Veal - Eye Of Round
20                         Peas - Frozen
21     Bar - Granola Trail Mix Fruit Nut
22                  Truffle Cups - Brown
23             Appetizer - Mushroom Tart
24            Br

0                         Pork - Kidney
1      Cheese - Boursin, Garlic / Herbs
2                  Lime Cordial - Roses
3                         Vanilla Beans
4         Ice Cream Bar - Hageen Daz To
5                        Fenngreek Seed
6                   Beets - Mini Golden
7                          Tahini Paste
8                   Thermometer Digital
9                  Coffee - Irish Cream
10               Coffee Decaf Colombian
11                        Chef Hat 20cm
12                    Tilapia - Fillets
13                     V8 - Berry Blend
14                 Lamb - Pieces, Diced
15                         Vol Au Vents
16                        Baking Powder
17                   Beef - Top Sirloin
18                               Pernod
19        Bread Crumbs - Japanese Style
20     Oil - Shortening - All - Purpose
21              Table Cloth 54x72 White
22                           Water, Tap
23           Potatoes - Idaho 100 Count
24           Wine - Crozes Hermitage E.


0             Smirnoff Green Apple Twist
1                  Rice - Jasmine Sented
2                                Pomello
3                        Skirt - 29 Foot
4                      Lettuce - Treviso
5                                 Pernod
6                 Towels - Paper / Kraft
7                      Garlic - Elephant
8                       Chocolate - Dark
9          Bread - Roll, Canadian Dinner
10                        Juice - Orange
11                   Initation Crab Meat
12                 Hickory Smoke, Liquid
13     Pasta - Cheese / Spinach Bauletti
14             Placemat - Scallop, White
15                  Cheese - Brie,danish
16      Wine - White, Schroder And Schyl
17                       Sausage - Liver
18                      Salsify, Organic
19                        Banana Turning
20                                  Kiwi
21                         Foam Cup 6 Oz
22             Tea - Herbal Sweet Dreams
23           Bouq All Italian - Primerba
24              

0                      Shrimp - 31/40
1                   Sprouts - Alfalfa
2       Wine - Blue Nun Qualitatswein
3                  Tomatoes Tear Drop
4                           Tia Maria
5      Chocolate - Semi Sweet, Calets
6                Dc Hikiage Hira Huba
7      Beef - Montreal Smoked Brisket
8        Cup - Translucent 7 Oz Clear
9                       Cheese - Wine
10                  Dc - Frozen Momji
11           Chestnuts - Whole,canned
12          Jolt Cola - Electric Blue
13                Wine - Red, Cooking
14             Pepper - White, Ground
15                 Pasta - Angel Hair
16     Bread - Italian Corn Meal Poly
17            Creme De Banane - Marie
18     Wine - Magnotta, Merlot Sr Vqa
19         Cookie Chocolate Chip With
20               Beans - Kidney White
21            Rum - Mount Gay Eclipes
22                  Veal - Osso Bucco
23          Ice Cream Bar - Oreo Cone
24                       Grapes - Red
25                Flavouring - Orange
26          

...

0              Chicken - Leg, Boneless
1                           Dried Figs
2            Carbonated Water - Cherry
3                          Beer - Blue
4        Wine - Vidal Icewine Magnotta
5      Squid - Tubes / Tenticles 10/20
6              Coffee - Hazelnut Cream
7                               Garlic
8                   Vinegar - Tarragon
9                 Truffle Cups - Brown
10                Pepsi - Diet, 355 Ml
11            Wine - White Cab Sauv.on
12                     Oil - Safflower
13                              Pernod
14                      Juice - Orange
15           Ice Cream Bar - Oreo Cone
16                        Blackberries
17                  Pasta - Angel Hair
18                        Vol Au Vents
19                  Scallops 60/80 Iqf
20                        Broom - Corn
21                 Flavouring - Orange
22             Table Cloth 81x81 White
23                  Lentils - Red, Dry
24                 Tray - 16in Rnd Blk
25                       

0                   Bread Fig And Almond
1          Bread Crumbs - Japanese Style
2                  Nantuket Peach Orange
3                          Wasabi Powder
4                 Pork - Loin, Bone - In
5                           Tahini Paste
6                       Halibut - Steaks
7         Carbonated Water - Blackcherry
8                           Broom - Corn
9                 Pants Custom Dry Clean
10        Pasta - Detalini, White, Fresh
11                 Cookie Dough - Double
12                  Beef - Prime Rib Aaa
13              Garlic - Primerba, Paste
14                    Beef - Top Sirloin
15        Chocolate - Semi Sweet, Calets
16                        Juice - Orange
17                         Vanilla Beans
18                Muffin - Zero Transfat
19          Wine - Charddonnay Errazuriz
20                     Assorted Desserts
21                    Peas - Pigeon, Dry
22                          Brandy - Bar
23               Crab - Imitation Flakes
24              

0               Chinese Foods - Chicken
1      Pail With Metal Handle 16l White
2                         Lamb - Ground
3      Cheese - Boursin, Garlic / Herbs
4                              Remy Red
5                        Fenngreek Seed
6                        Spinach - Baby
7            Rosemary - Primerba, Paste
8                  Coffee - Irish Cream
9                         Chef Hat 20cm
10         Chocolate - Compound Coating
11                     V8 - Berry Blend
12              Soupcontfoam16oz 116con
13                 Lettuce - Spring Mix
14                  Cheese - Mozzarella
15             Cheese - Parmesan Grated
16                         Sugar - Fine
17                       Mustard - Seed
18                               Pernod
19              Soupfoamcont12oz 112con
20             Wine - White Cab Sauv.on
21        Bar Mix - Pina Colada, 355 Ml
22           Wine - Chablis 2003 Champs
23                            Milk - 1%
24        Longos - Grilled Chicken With


0               Bread - French Baquette
1            Spice - Peppercorn Melange
2                         Sauce - Rosee
3                     Rice - Long Grain
4                  Salmon - Sockeye Raw
5      Wine - White, Schroder And Schyl
6                 Cookie Dough - Double
7                     Zucchini - Yellow
8                 Oregano - Dry, Rubbed
9                     Veal - Osso Bucco
10                  Garbag Bags - Black
11                   Bread - Multigrain
12                 Garbage Bags - Clear
13                       Fond - Neutral
14                      Extract - Lemon
15                          Fuji Apples
16                Kellogs All Bran Bars
17                        Puree - Mocha
18                  Juice - Apple Cider
19          Mayonnaise - Individual Pkg
20                         Brandy - Bar
21                 Dc Hikiage Hira Huba
22                      Wonton Wrappers
23         Langers - Ruby Red Grapfruit
24       French Pastry - Mini Chocolate


0                 Pants Custom Dry Clean
1      Bar - Granola Trail Mix Fruit Nut
2                 Muffin Batt - Choc Chk
3            Water - Green Tea Refresher
4            Mayonnaise - Individual Pkg
5            Pepper - Paprika, Hungarian
6            Oil - Shortening,liqud, Fry
7                           Broom - Corn
8          Wine - Prosecco Valdobiaddene
9          Longos - Grilled Chicken With
10            Wine - Two Oceans Cabernet
11                 Nantuket Peach Orange
12          Fish - Scallops, Cold Smoked
13                         Sauce - Rosee
14                            Watercress
15                      Mussels - Frozen
16                         Sage - Ground
17                           Guinea Fowl
18                         Hersey Shakes
19            Yogurt - Blueberry, 175 Gr
20                  Garbage Bags - Clear
21       Wine - Red, Harrow Estates, Cab
22                           Fuji Apples
23                          Pears - Bosc
24              

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [ ]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.